In [1]:
!pip install kagglehub[pandas-datasets]

In [2]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [1]:
!pip install kaggle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install ijson

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 9.6 MB/s eta 0:00:00


Fetching partial data

In [1]:
import ijson
import pandas as pd

# Set the path to the file in your Google Drive
file_path = '/content/drive/My Drive/mined-comments-25stars-25prs-Go.json'  # Update this path with your actual file location

# Initialize a list to hold data incrementally
data = []

# Open the JSON file and parse incrementally
with open(file_path, 'r') as f:

    # We start by parsing the 'root' object
    parser = ijson.parse(f)

    # Store the current repository name
    current_repo = None
    current_items = []

    # Iterate over the parsed JSON objects
    for prefix, event, value in parser:
        # If we encounter the root key, we know we are inside the 'root' object
        # print(f"prefix: {prefix}, event: {event}, value: {value}")
        if prefix == 'root':
            continue
        # print(value)
        # If the key is a repository (e.g., 'golang/go', 'python/cpython')
        if event == 'map_key' and current_repo == None:
            # We found a repository name, so store it
            current_repo = value
            current_item = {}

        # When we encounter a key in the comment object, just store it
        elif event == 'map_key' and current_repo:
            # For each key inside the comment object, just store it as part of the comment data
            current_item['repo'] = current_repo  # Add repository info
            current_item['key'] = value  # Store the key (optional

        elif (event == 'string' or event == 'number') and current_repo:
            # print("EVENTVAL")
            # Now, we store the value for the corresponding key
            key = current_item.get('key')  # Retrieve the most recent key
            if key:
                # If the current_item already has this key, append the value to it
                current_item[key] = value
                current_item.pop('key', None)  # Remove the key from the current_item
                # For debugging: print the key-value pair being stored
                # print(f"Storing key-value: {key} = {value}")

        # When we reach the end of an array, clear current_items (optional)
        elif event == 'end_array' and current_repo:
            if current_item != {}:
              data.append(current_item)
              # print(f"Found item: {current_item}")
            current_item = {}
            current_repo = None

# Convert the list of records to a DataFrame for better viewing or further processing
df = pd.DataFrame(data)

# Optionally, print the first 5 rows using pandas
print("DataFrame (First 5 rows):")
print(df.head())

DataFrame (First 5 rows):
                    repo                                           html_url  \
0              golang/go  https://github.com/golang/go/pull/59301#discus...   
1     avelino/awesome-go  https://github.com/avelino/awesome-go/pull/440...   
2          gin-gonic/gin  https://github.com/gin-gonic/gin/pull/3556#dis...   
3  kubernetes/kubernetes  https://github.com/kubernetes/kubernetes/pull/...   
4           fatedier/frp  https://github.com/fatedier/frp/pull/3404#disc...   

                            path    line  \
0             src/errors/join.go    14.0   
1                   make_site.go    13.0   
2                     context.go  1059.0   
3  pkg/registry/etcd_registry.go   185.0   
4              client/service.go   461.0   

                                                body             user  \
0  Would it be appropriate to be more explicit he...          dnephin   
1  Sure, it does. That's why it is still used and...        benschoch   
2  @fabricioere

In [2]:
print("Number of entries (rows) in the DataFrame:", df.shape[0])

Number of entries (rows) in the DataFrame: 7154


In [14]:
# Assuming df is your DataFrame

# Write the DataFrame to a JSON file
df.to_json('/content/drive/My Drive/output_data.json', orient='records', lines=True)

In [17]:
import json

# Convert the DataFrame to a list of records
records = df.to_dict(orient='records')

# Write to a file without escaping the slashes
with open('/content/drive/My Drive/public_pr.json', 'w') as f:
    json.dump(records, f, separators=(',', ':'), ensure_ascii=False)

In [13]:
import os
print(os.getcwd())

/content


Fetching entire data (3100000+ entries)

In [1]:
import ijson
import pandas as pd

# Set the path to the file in your Google Drive
file_path = '/content/drive/My Drive/mined-comments-25stars-25prs-Go.json'  # Update this path with your actual file location

# Initialize a list to hold data incrementally
data = []

# Open the JSON file and parse incrementally
with open(file_path, 'r') as f:

    # We start by parsing the 'root' object
    parser = ijson.parse(f)

    # Store the current repository name
    current_repo = None
    current_item = None  # This will hold individual comment data
    current_items = []

    # Iterate over the parsed JSON objects
    for prefix, event, value in parser:
        # If we encounter the root key, we know we are inside the 'root' object
        # print(f"Event: {event}, Prefix: {prefix}, Value: {value}")  # Debugging output

        if prefix == 'root':
            continue
        # print(value)
        # If the key is a repository (e.g., 'golang/go', 'python/cpython')
        if event == 'map_key' and current_repo == None:
            # We found a repository name, so store it
            current_repo = value
            # current_item = {}
            continue

        elif event == 'start_array' and current_repo:
            # Initialize the current item as an empty dictionary to store comment data
            # print("Starting array")
            current_item = {'repo': current_repo}  # Initialize with repository name
            continue

        elif event == 'start_map':
            # print("Starting map")
            current_item = {'repo': current_repo}

        # When we encounter a key in the comment object, just store it
        elif event == 'map_key' and current_item:
            # For each key inside the comment object, just store it as part of the comment data
            # print(current_item
            current_item['key'] = value  # Store the key (optional

        elif (event == 'string' or event == 'number') and current_item:
            # print("EVENTVAL")
            # Now, we store the value for the corresponding key
            key = current_item.get('key')  # Retrieve the most recent key
            if key:
                # If the current_item already has this key, append the value to it
                current_item[key] = value
                current_item.pop('key', None)  # Remove the key from the current_item
                # For debugging: print the key-value pair being stored
                # print(f"Storing key-value: {key} = {value}")

        elif event == 'end_map' and current_item:
            # Once the item is fully processed, append it to the data list
            data.append(current_item)
            if (len(data)%100000 == 0):
                print("Processed ", len(data))
            # print(f"Found item: {current_item}")  # For debugging, print the item
            current_item = None

        # When we reach the end of an array, clear current_items (optional)
        elif event == 'end_array' and current_repo:
            # print("Ending array")
            current_repo = None

# Convert the list of records to a DataFrame for better viewing or further processing
df_complete = pd.DataFrame(data)

Processed  100000
Processed  200000
Processed  300000
Processed  400000
Processed  500000
Processed  600000
Processed  700000
Processed  800000
Processed  900000
Processed  1000000
Processed  1100000
Processed  1200000
Processed  1300000
Processed  1400000
Processed  1500000
Processed  1600000
Processed  1700000
Processed  1800000
Processed  1900000
Processed  2000000
Processed  2100000
Processed  2200000
Processed  2300000
Processed  2400000
Processed  2500000
Processed  2600000
Processed  2700000
Processed  2800000
Processed  2900000
Processed  3000000
Processed  3100000


In [4]:
# df_complete.to_json('/content/drive/My Drive/public_pr_complete.json', orient='records', lines=True)

import json

# Convert the DataFrame to a list of records
# records = df_complete.to_dict(orient='records')
chunk_size = 50000  # Set chunk size to a manageable number, adjust as necessary

# Write to a file without escaping the slashes
with open('/content/drive/My Drive/public_pr_complete.json', 'w') as f:
    for start in range(0, len(df_complete), chunk_size):
        end = min(start + chunk_size, len(df_complete))
        chunk = df_complete.iloc[start:end]
        records = chunk.to_dict(orient='records')
        json.dump(records, f, separators=(',', ':'), ensure_ascii=False)
        # Convert chunk to JSON and write it to the file
        # chunk.to_json(f, orient='records', lines=True, header=(start == 0))

        # Optionally, add a separator between chunks (if needed)
        f.write('\n')  # This is only necessary if you're appending


In [5]:
df_complete.to_csv('/content/drive/My Drive/public_pr_complete.csv', index=False)

In [2]:
!pip install jsonlines

In [9]:
import jsonlines

# Write to a jsonlines file without escaping the slashes
# with jsonlines.open('/content/drive/My Drive/public_pr_complete.json', mode='w') as writer:
chunk_size = 1000000
for start in range(0, len(df_complete), chunk_size):
    end = min(start + chunk_size, len(df_complete))
    print(f"Reached {end}")
    chunk = df_complete.iloc[start:end]
    records = chunk.to_dict(orient='records')
    with jsonlines.open('/content/drive/My Drive/public_pr_complete'+str(end//chunk_size)+'.json', mode='w') as writer:
      writer.write_all(records)  # Write all records in this chunk to the file
      break

print("DataFrame saved successfully using jsonlines without escaped slashes.")

Reached 3000000
DataFrame saved successfully using jsonlines without escaped slashes.


Sampling 10k random entries

In [5]:
df_sampled = df_complete.sample(n=10000, random_state=42)

In [5]:
print("Number of entries (rows) in the DataFrame:", df_sampled.shape[0])

Number of entries (rows) in the DataFrame: 10000


In [6]:
print(df_sampled.iloc[0, 6])

@@ -0,0 +1,35 @@
+package main
+
+import (
+	"bytes"
+	"io"
+	"os"
+	"os/exec"
+	"testing"
+)
+
+func TestLoginWithoutTTY(t *testing.T) {
+	cmd := exec.Command(dockerBinary, "login")
+	// setup STDOUT and STDERR so that we see any output and errors in our console
+	cmd.Stdout = os.Stdout
+	cmd.Stderr = os.Stderr
+
+	// create a buffer with text then a new line as a return
+	buf := bytes.NewBuffer([]byte("buffer test string \n"))


In [4]:
print(df_complete.iloc[0])

repo                                                          golang/go
html_url              https://github.com/golang/go/pull/23805#discus...
path                                              src/math/rand/rand.go
line                                                              213.0
body                  While `true` and `false` is not specified as a...
user                                                             mewmew
diff_hunk             @@ -210,6 +210,11 @@ again:\n \treturn f\n }\n...
author_association                                          CONTRIBUTOR
commit_id                      f200cd75ab7c3fd16e046fa3cbc76565c4063cec
id                                                            178204099
Name: 0, dtype: object


In [18]:
df_sampled_v1 = df_complete.sample(n=10000, random_state=12)
# print(df_sampled_v1)
unique_count = df_sampled_v1['html_url'].nunique()
print(unique_count)

10000


In [26]:
import json

# Convert the DataFrame to a list of records
records = df_sampled_v1.to_dict(orient='records')

# Write to a file without escaping the slashes
with open('/content/drive/My Drive/public_pr_sampled.json', 'w') as f:
    json.dump(records, f, separators=(',', ':'), ensure_ascii=False)

Sampling 1k entries

In [40]:
df_sampled_v2 = df_complete.sample(n=1000, random_state=42)
# print(df_sampled_v1)
unique_count = df_sampled_v2['html_url'].nunique()
print(unique_count)

1000


Grouping same PR entries

In [41]:
# df_filtered = df_complete[df_complete['html_url'].isin(df_sampled_v1['html_url'])]
df_filtered = df_complete[df_complete['html_url'].str.startswith(tuple(df_sampled_v2['html_url']))]
print(df_filtered.shape[0])

1002


In [51]:
# Aggregate comments
df_grouped = df_filtered.groupby('html_url').agg({'body': lambda x: ' | '.join(x)}).reset_index()
print(df_grouped.shape[0])
print(df_grouped.iloc[0])

1000
html_url    https://github.com/0chain/0chain/pull/1258#dis...
body        do we ever query "all challenges for current v...
Name: 0, dtype: object


Fetching pr details including title, description and code diff for the sampled 1k

In [ ]:
# get pr details
import requests
from urllib.parse import urlparse

def get_pr_details(pr_url):
    """Fetch PR title, description, and code diff from GitHub API."""
    GITHUB_TOKEN = "<redacted>"
    headers = {"Authorization": f"token {GITHUB_TOKEN}"}

    # Extract repo owner, repo name, and PR number from the URL
    parsed_url = urlparse(pr_url)
    clean_url = parsed_url._replace(fragment="").geturl()  # Remove fragment

    # Split the cleaned URL into parts
    parts = clean_url.rstrip('/').split('/')

    # Extract owner, repo, and PR number
    if len(parts) < 5:  # Ensure URL is correctly structured
        return None, None, None

    owner, repo, pr_number = parts[-4], parts[-3], parts[-1]
    # GitHub API URL
    api_url = f"https://api.github.com/repos/{owner}/{repo}/pulls/{pr_number}"
    diff_url = f"https://github.com/{owner}/{repo}/pull/{pr_number}.diff"
    # print(diff_url)

    try:
        # Fetch PR metadata (title & description)
        response = requests.get(api_url, headers=headers)
        response.raise_for_status()  # Raise an error for bad requests
        pr_data = response.json()

        title = pr_data.get('title', 'N/A')
        description = pr_data.get('body', 'No description')

        # Fetch PR diff
        diff_response = requests.get(diff_url, headers=headers)
        diff_response.raise_for_status()
        code_diff = diff_response.text[:]  # Limit diff size to avoid long outputs

        return title, description, code_diff

    except requests.exceptions.RequestException as e:
        print(f"Error fetching PR {pr_number}: {e}")
        return "N/A", "N/A", "N/A"

# Example usage
pr_url = "https://github.com/moby/moby/pull/9211#discus"
title, description, code_diff = get_pr_details(pr_url)
print(f"Title: {title}\nDescription: {description}\nDiff: {code_diff}")


Title: Fixed a bug - no panic anymore when logining in without TTY
Description: Fixes #8956

Signed-off-by: Marianna mtesselh@gmail.com

Diff: diff --git a/api/client/commands.go b/api/client/commands.go
index 6b9c4d4d8fbb5..dfc07835c9016 100644
--- a/api/client/commands.go
+++ b/api/client/commands.go
@@ -289,7 +289,10 @@ func (cli *DockerCli) CmdLogin(args ...string) error {
 	// the password or email from the config file, so prompt them
 	if username != authconfig.Username {
 		if password == "" {
-			oldState, _ := term.SaveState(cli.inFd)
+			oldState, err := term.SaveState(cli.inFd)
+			if err != nil {
+				return err
+			}
 			fmt.Fprintf(cli.out, "Password: ")
 			term.DisableEcho(cli.inFd, oldState)
 
diff --git a/integration-cli/docker_cli_login_test.go b/integration-cli/docker_cli_login_test.go
new file mode 100644
index 0000000000000..cf134e4c9b39c
--- /dev/null
+++ b/integration-cli/docker_cli_login_test.go
@@ -0,0 +1,35 @@
+package main
+
+import (
+	"bytes"
+	"io"
+	"os"

In [61]:
df_grouped[['pr_title', 'pr_description', 'pr_diff']] = df_grouped['html_url'].apply(
    lambda url: pd.Series(get_pr_details(url))
)

# Reorder columns for readability
df_final = df_grouped[['pr_title', 'pr_description', 'html_url', 'body', 'pr_diff']]

Error fetching PR 689: 404 Client Error: Not Found for url: https://api.github.com/repos/CodeIntelligenceTesting/cifuzz/pulls/689
Error fetching PR 2327: HTTPSConnectionPool(host='github.com', port=443): Max retries exceeded with url: /huaweicloud/terraform-provider-huaweicloud/pull/2327.diff (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1006)')))
Error fetching PR 412: 404 Client Error: Not Found for url: https://api.github.com/repos/idcos/cloudiac/pulls/412
Error fetching PR 580: 404 Client Error: Not Found for url: https://api.github.com/repos/idcos/cloudiac/pulls/580
Error fetching PR 1352: 404 Client Error: Not Found for url: https://api.github.com/repos/incognitochain/incognito-chain/pulls/1352
Error fetching PR 253: 404 Client Error: Not Found for url: https://api.github.com/repos/kiegroup/kogito-operator/pulls/253
Error fetching PR 4463: 404 Client Error: Not Found for url: https://api.github.com/repositori

In [62]:
print(df_final.shape[0])
print(df_final.head(5))

1000
                                            pr_title  \
0                                    challenge table   
1  fix(tokenrate): fixed json issue for bancor ap...   
2                                          Syncer V2   
3                                 Proposer selection   
4  zeroex,ethereum,meshdb: Add support for valida...   

                                      pr_description  \
0  ## Fixes\r\n- removed PrevID from code\r\n\r\n...   
1  ### Changes\r\n-\r\n\r\n### Fixes\r\n- fixed #...   
2  # Description\r\n\r\nImplementation of the new...   
3  # Description\r\n\r\nImplementation of propose...   
4  Fixes: https://github.com/0xProject/0x-mesh/pu...   

                                            html_url  \
0  https://github.com/0chain/0chain/pull/1258#dis...   
1  https://github.com/0chain/gosdk/pull/592#discu...   
2  https://github.com/0xPolygon/polygon-edge/pull...   
3  https://github.com/0xPolygon/polygon-edge/pull...   
4  https://github.com/0xProject/0x-mesh/p

In [63]:
import json

# Convert the DataFrame to a list of records
records = df_final.to_dict(orient='records')

# Write to a file without escaping the slashes
with open('/content/drive/My Drive/public_pr_final.json', 'w') as f:
    json.dump(records, f, separators=(',', ':'), ensure_ascii=False)

# Experimenting with LLM

In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `pr-review` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `pr-review`


Experimenting with LLAMA but due to insuffienct GPU resources couldn't complete execution.

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "meta-llama/Meta-Llama-3-8B" #"meta-llama/Meta-Llama-3-8B"  # Change to "mistralai/Mistral-7B-v0.1" or "codellama/CodeLlama-7b-Python"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [4]:

def review_pr(title, description, code_diff):
    prompt = f"""
    You are a senior software engineer reviewing a pull request.

    **PR Title:** {title}
    **PR Description:** {description}
    **Code Diff:**
    ```
    {code_diff}
    ```

    Please review this PR and provide:
    - Feedback on code quality
    - Any potential issues or bugs
    - Suggestions for improvement
    - Whether it follows best practices

    Provide a structured response.
    """
    inputs = tokenizer(prompt, return_tensors="pt").to("cpu")
    output = model.generate(**inputs, max_new_tokens=300)
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [7]:
inputs = tokenizer("What is AI?", return_tensors="pt").to("cpu")
output = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


What is AI? And How Is It Being Used in the Modern World?
Artificial Intelligence (AI) is a term that has been around for decades, but it has recently gained popularity as technology advances and the world becomes more connected. AI is a branch of computer science


Exploring dataset

In [9]:
duplicates_exist = df_complete['html_url'].duplicated().any()
print(duplicates_exist)

True


In [20]:
print(df_complete['html_url'].head())
filtered_df = df_complete[df_complete['html_url'].str.startswith('https://github.com/moby/moby/pull/9211#discus')]
print(filtered_df)
print(df_complete['html_url'].eq('https://github.com/golang/go/pull/23805#discus').any())

0    https://github.com/golang/go/pull/23805#discus...
1    https://github.com/golang/go/pull/27971#discus...
2    https://github.com/golang/go/pull/28420#discus...
3    https://github.com/golang/go/pull/28420#discus...
4    https://github.com/golang/go/pull/28624#discus...
Name: html_url, dtype: object
            repo                                           html_url  \
13686  moby/moby  https://github.com/moby/moby/pull/9211#discuss...   

                                           path  line  \
13686  integration-cli/docker_cli_login_test.go  18.0   

                                                    body   user  \
13686  I think all code until `cmd.Run` can be just `...  LK4D4   

                                               diff_hunk author_association  \
13686  @@ -0,0 +1,35 @@\n+package main\n+\n+import (\...        CONTRIBUTOR   

                                      commit_id        id  
13686  e6fd57b90bcdd3b0105fa5c7d17a64332f829ded  20485775  
False


In [10]:
print(df_complete.columns)

Index(['repo', 'html_url', 'path', 'line', 'body', 'user', 'diff_hunk',
       'author_association', 'commit_id', 'id'],
      dtype='object')
